## importing relavent libraries

In [2]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

import os
import wget
import time

sns.set()

## setting up chrome driver

In [4]:
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.elmenus.com/cairo/delivery/nasr-city/features-order-online')

C:\Users\Repoo\AppData\Local\Temp/ipykernel_6472/3774652601.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


## website processing

In [5]:
#restaurant-card restaurant-delivery-card col-md-5 col-sm-8 col-xs-16

html        = bs(driver.page_source, 'lxml')
tags_slider = html.find('div', attrs={'id': 'tags-slider'})
top_dishes  = tags_slider.find('div', attrs={'class': 'glider-track'})
top_dishes  = [i.text.replace('\n          ', '').strip().lower().strip("&").replace(" ", "-") for i in top_dishes]
top_dishes = top_dishes[1:]

## scraping the website and store the data in a dataframe

In [6]:
df = pd.DataFrame([], columns=['name', 'image_url', 'description', 'likes', 'section_title', 'res_name', 'res_type', 'res_href', 'offer', 'delivery_time', 'res_ratting', 'res_reviews'])

In [7]:
#get res based on res type on top dishes

for i in top_dishes:
    driver.get('https://www.elmenus.com/cairo/delivery/nasr-city/dish-' + i + '/features-order-online')
    while True:
        try:
            load_more = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='btn btn-primary load-more-btn']"))).click()
            time.sleep(3)
        except:
            print('done')
            break
    #elmenus[i] = {}
    html = bs(driver.page_source, 'lxml')
    restaurants = html.find_all('div', attrs={'class': 'restaurant-card restaurant-delivery-card col-md-5 col-sm-8 col-xs-16'})
    for j, res in enumerate(restaurants):
        res_title = restaurants[j].find('h3', class_='card-title').string.strip()
        res_href = restaurants[j].find('h3', class_='card-title').find('a')['href']
        try:
            offer = restaurants[j].find('div', class_='offer-text').string.strip()
        except:
            offer = 'NA'
        delivery_time = restaurants[j].find('div', class_='card-footer').find('span').text.strip().replace('\n\n     ', '')
        dishes  = []

        driver.get('https://www.elmenus.com' + res_href)
        time.sleep(3)
        res_page = bs(driver.page_source, 'lxml')
        cat_sections = res_page.find_all('div', class_='cat-section')
        try:
            res_ratting = res_page.find('span', class_='vue-star-rating-rating-text').text.strip()
        except:
            res_ratting = 0
        try:
            res_reviews = res_page.find('a', class_='reviews').text.strip().replace("( ", '').replace(" )", "")
        except:
            res_reviews = 0
            
        for k, sec in enumerate(cat_sections):
            try:
                section_title = cat_sections[k].find('h2', class_='section-title').text.strip()
            except:
                section_title = cat_sections[k].find('h3', class_='section-title').text.strip().replace('\n                  ', '')
            sec_body = cat_sections[k].find('div', class_="section-body").find_all('div', recursive=False)
            
            for m in sec_body:
                name = m.find('h5', class_='title').text.strip()
                try:
                    image_url = m.find('img')['data-src']
                except:
                    image_url = 'NA'
                try:
                    description = m.find('p', class_='description').text.strip()
                except:
                    description = 'NA'
                try:
                    price = m.find('p', class_='price').find('span', class_='bold').text.strip().replace('\n                     ', '')
                except:
                    price = 0
                try:
                    likes = m.find('span', class_='count').text.strip()
                except:
                    likes = 0
                
                df.loc[-1] = [name, image_url, description, likes, section_title, res_title, i, res_href, offer, delivery_time, res_ratting, res_reviews]
                df.index = df.index + 1
                df = df.sort_index()

## create a csv file with the scrapped data

In [1]:
df.to_csv('elmenus_new.csv')

NameError: name 'df' is not defined